In [1]:
import pandas as pd
import sys
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict, KFold
from sklearn.metrics import roc_auc_score
import csv
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from optparse import OptionParser

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#clicks_train = pd.read_csv("/home/nisha/Outbrain/clicks_train.csv")
X_test=pd.read_csv("/home/nisha/Outbrain/clicks_test.csv")
documents_topics_df =pd.read_csv("/home/nisha/Outbrain/documents_topics.csv")


In [31]:
X_test.shape

(32225162, 2)

In [3]:
documents_topics_df.drop("confidence_level", axis=1, inplace=True)
documents_entities_df = pd.read_csv('/home/nisha/Outbrain/documents_entities.csv')
documents_entities_df.drop("confidence_level", axis=1, inplace=True)

In [4]:
documents_meta_df = pd.read_csv('/home/nisha/Outbrain/documents_meta.csv')


In [5]:
#page_views_sample_df = pd.read_csv('/home/nisha/Outbrain/page_views_sample.csv')

In [25]:
#events_df = pd.read_csv('/home/nisha/Outbrain/events.csv')

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
promoted_content_df = pd.read_csv('/home/nisha/Outbrain/promoted_content.csv')

In [6]:

#documents_topics_df = documents_topics_df.drop_duplicates(subset='document_id', keep='first')
print(any(promoted_content_df['ad_id'].duplicated())) 

False


In [5]:
promoted_content_df.shape

(559583, 4)

In [7]:
documents_categories_df = pd.read_csv('/home/nisha/Outbrain/documents_categories.csv')

In [39]:
#Add the calculated scores to promoted_content based on 'ad_id'
topics_promoted=documents_topics_df.merge(documents_meta_df,on='document_id' ,how='inner')

In [42]:
#topics_promoted = topics_promoted.drop_duplicates(subset=['document_id','ad_id'], keep='first')
print(any(topics_promoted['document_id'].duplicated())) 

(2495423, 6)

In [43]:
del documents_topics_df, documents_meta_df

In [47]:
documents_categories_df = documents_categories_df.drop_duplicates(subset='document_id', keep='first')
print(any(documents_categories_df['document_id'].duplicated())) 

False


In [48]:

topic_doc_cat_df = topics_promoted.merge(documents_categories_df,on='document_id' ,how='left')


In [49]:
topic_doc_cat_df.shape

(2495423, 8)

In [50]:
del documents_categories_df, topics_promoted

In [54]:
promoted_content_df.shape

(559583, 4)

In [55]:
topic_doc_cat_ad_df = topic_doc_cat_df.merge(promoted_content_df,on='document_id' ,how='inner')

In [56]:
topic_doc_cat_ad_df.shape

(535125, 11)

In [57]:
topic_doc_cat_ad_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535125 entries, 0 to 535124
Data columns (total 11 columns):
document_id           535125 non-null int64
topic_id              535125 non-null int64
confidence_level_x    535125 non-null float64
source_id             533718 non-null float64
publisher_id          256039 non-null float64
publish_time          402205 non-null object
category_id           534814 non-null float64
confidence_level_y    534814 non-null float64
ad_id                 535125 non-null int64
campaign_id           535125 non-null int64
advertiser_id         535125 non-null int64
dtypes: float64(5), int64(5), object(1)
memory usage: 49.0+ MB


In [58]:
topic_doc_cat_ad_df.drop(["campaign_id","advertiser_id", "publish_time", "confidence_level_x", "confidence_level_y"], axis=1)

,document_id,topic_id,source_id,publisher_id,category_id,ad_id
0,1009570,232,2.0,85.0,1100.0,61909
1,1097501,232,2.0,85.0,1100.0,90151
2,1097501,232,2.0,85.0,1100.0,90152
3,1097501,232,2.0,85.0,1100.0,90153
4,1097501,232,2.0,85.0,1100.0,90154
5,1484804,232,2.0,85.0,1100.0,207643
6,1324010,232,2.0,85.0,1100.0,141241
7,1142982,232,2.0,85.0,1100.0,140507
8,1177339,77,2.0,85.0,1100.0,138386
9,1177339,77,2.0,85.0,1100.0,138451


In [59]:

print(any(topic_doc_cat_ad_df['ad_id'].duplicated())) 

False


In [26]:


#topic_doc_cat_src_plt_df = topic_doc_cat_src_df.merge(events_df,on='document_id' ,how='inner')


In [29]:
#del topic_doc_cat_src_plt_df

In [60]:
test_merged = X_test.merge(topic_doc_cat_ad_df, on='ad_id' ,how='left')

In [61]:
test_merged.shape

(32225162, 12)

In [63]:
test_merged.to_csv("train_merged.csv", sep=',')
print("saved")


saved


In [34]:
clicks_train.info()
clicks_train.iloc[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87141731 entries, 0 to 87141730
Data columns (total 6 columns):
display_id       int64
ad_id            int64
clicked          int64
document_id      int64
campaign_id      int64
advertiser_id    int64
dtypes: int64(6)
memory usage: 4.5 GB


display_id            1
ad_id             42337
clicked               0
document_id      938164
campaign_id        5969
advertiser_id      1499
Name: 0, dtype: int64

In [111]:
del clicks_train

In [35]:
# join documents_topics and documents_entities by document_id
by_document_id_df = documents_topics_df.merge(documents_entities_df, on='document_id', how='inner')
by_document_id_df.rename(columns={'confidence_level_x': 'topic_confidence_level', 'confidence_level_y': 'entities_confidence_level'}, inplace=True)


document_id                                  1524246
topic_id                                         113
confidence_level                             0.19645
entity_id           f9eec25663db4cd83183f5c805186f16
Name: 0, dtype: object

In [36]:
by_document_id_df.info()
by_document_id_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33360919 entries, 0 to 33360918
Data columns (total 4 columns):
document_id         int64
topic_id            int64
confidence_level    float64
entity_id           object
dtypes: float64(1), int64(2), object(1)
memory usage: 1.2+ GB


document_id                                  1524246
topic_id                                         113
confidence_level                             0.19645
entity_id           f9eec25663db4cd83183f5c805186f16
Name: 0, dtype: object

In [37]:
# join by_document_id_df with documents_meta_df by document_id
by_document_id_df = by_document_id_df.merge(documents_meta_df, on="document_id", how="inner")

# join by_document_id_df with documents_categories_df by document_id
by_document_id_df = by_document_id_df.merge(documents_categories_df, on="document_id", how="inner")
by_document_id_df.rename(columns={'confidence_level': 'category_confidence_level'}, inplace=True)

In [38]:
by_document_id_df.info()
by_document_id_df.iloc[0]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66003848 entries, 0 to 66003847
Data columns (total 8 columns):
document_id                  int64
topic_id                     int64
category_confidence_level    float64
entity_id                    object
source_id                    float64
publisher_id                 float64
publish_time                 object
category_id                  int64
dtypes: float64(3), int64(3), object(2)
memory usage: 4.4+ GB


document_id                                           1524246
topic_id                                                  113
category_confidence_level                             0.19645
entity_id                    f9eec25663db4cd83183f5c805186f16
source_id                                                   1
publisher_id                                              603
publish_time                              2016-05-26 11:00:00
category_id                                              1807
Name: 0, dtype: object

In [39]:
# join page_views_sample with events by uuid, document_id, timestamp, platform, geo_location
join_columns = ["uuid", "document_id", "timestamp", "platform", "geo_location"]
page_views_sample_events_df = page_views_sample_df.merge(events_df, left_on=join_columns, right_on=join_columns, how="inner")


In [40]:
page_views_sample_events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70665 entries, 0 to 70664
Data columns (total 7 columns):
uuid              70665 non-null object
document_id       70665 non-null int64
timestamp         70665 non-null int64
platform          70665 non-null object
geo_location      70665 non-null object
traffic_source    70665 non-null int64
display_id        70665 non-null int64
dtypes: int64(4), object(3)
memory usage: 4.3+ MB


In [41]:
# join by_document_id_df with page_views_sample_events_df by document_id
by_document_id_with_page_views_sample_events_df = by_document_id_df.merge(page_views_sample_events_df, on="document_id", how="inner")


In [59]:
by_document_id_with_page_views_sample_events_df.iloc[0]

document_id                                           1587682
topic_id                                                  260
category_confidence_level                            0.179229
entity_id                    192400be544694adc47d343f46a7117c
source_id                                                   1
publisher_id                                              603
publish_time                              2016-06-09 00:00:00
category_id                                              1603
uuid                                           830ad80b58113f
timestamp                                              551806
platform                                                    2
geo_location                                        US>CO>751
traffic_source                                              1
display_id                                               7998
Name: 0, dtype: object

In [61]:
# join above with clicks_train_df and clicks_test_df by display_id
train_df = by_document_id_with_page_views_sample_events_df.merge(clicks_train, on="display_id", how="inner")

In [47]:

#test_df = by_document_id_with_page_views_sample_events_df.merge(X_test, left_on="display_id", right_on="display_id", how="inner")
test_df = X_test.merge(by_document_id_with_page_views_sample_events_df, on="display_id", how="left")



In [63]:
# drop entity_id, publish_time, uuid, timestamp
columns_to_drop = ["entity_id", "publish_time", "uuid", "timestamp"]
train_df = train_df.drop(columns_to_drop, axis=1)
test_df = test_df.drop(columns_to_drop, axis=1)

In [81]:
test_df.drop_duplicates()
print(test_df.shape)

(33799595, 11)


In [26]:
test_df.iloc[0]

display_id        16874594
ad_id                66758
document_id            NaN
topic_id               NaN
source_id              NaN
publisher_id           NaN
category_id            NaN
platform               NaN
geo_location           NaN
traffic_source         NaN
Name: 0, dtype: object

In [14]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33799595 entries, 0 to 33799594
Data columns (total 10 columns):
display_id        int64
ad_id             int64
document_id       float64
topic_id          float64
source_id         float64
publisher_id      float64
category_id       float64
platform          object
geo_location      object
traffic_source    float64
dtypes: float64(6), int64(2), object(2)
memory usage: 2.8+ GB


In [65]:
train_df.shape


(8377862, 12)

In [68]:
# save train and test data to file
train_df.to_csv("train_final.csv", sep=',')
print("saved train data to file")

train_df.to_csv("test_final.csv", sep=',')
print("saved test data to file")

print("Finished loading files")

saved train data to file
saved test data to file
Finished loading files


In [65]:
train_final_df = pd.read_csv("train_final.csv")

In [67]:
train_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8377862 entries, 0 to 8377861
Data columns (total 13 columns):
Unnamed: 0                   int64
document_id                  int64
topic_id                     int64
category_confidence_level    float64
source_id                    float64
publisher_id                 float64
category_id                  int64
platform                     int64
geo_location                 object
traffic_source               int64
display_id                   int64
ad_id                        int64
clicked                      int64
dtypes: float64(3), int64(9), object(1)
memory usage: 830.9+ MB


In [68]:
features_columns = ["document_id", "topic_id", "source_id", "publisher_id", 
                    "category_id", "ad_id", "display_id"]

X = train_final_df[features_columns]
Y = train_final_df["clicked"]


In [76]:
test_final_df = pd.read_csv("train_merged.csv")

In [69]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8377862 entries, 0 to 8377861
Data columns (total 7 columns):
document_id     int64
topic_id        int64
source_id       float64
publisher_id    float64
category_id     int64
ad_id           int64
display_id      int64
dtypes: float64(2), int64(5)
memory usage: 447.4 MB


In [78]:
test_final_df.drop(test_final_df.columns[0],axis=1,inplace=True)

In [79]:
test_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32225162 entries, 0 to 32225161
Data columns (total 12 columns):
display_id            int64
ad_id                 int64
document_id           float64
topic_id              float64
confidence_level_x    float64
source_id             float64
publisher_id          float64
publish_time          object
category_id           float64
confidence_level_y    float64
campaign_id           float64
advertiser_id         float64
dtypes: float64(9), int64(2), object(1)
memory usage: 2.9+ GB


In [70]:
rf = RandomForestClassifier(random_state=1, class_weight="balanced", n_estimators=25, max_depth=6)
rf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=25, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [80]:
features_columns = ["document_id", "topic_id", "source_id", "publisher_id", 
                    "category_id", "ad_id", "display_id"]

X_test = test_final_df[features_columns]

In [84]:

predictions_proba = rf.predict_proba(X_test)



In [87]:
predictions = rf.predict(X_test)

In [89]:
X_test["predicted_label"] = predictions


/home/nish_sharon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [95]:
X_test["predictions_proba"] = predictions_proba.T[1]

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [94]:
predictions_proba.T[1]

array([ 0.47923445,  0.49374897,  0.53516155, ...,  0.50910103,
        0.47786933,  0.45180389])

In [96]:
X_test.columns

Index(['document_id', 'topic_id', 'source_id', 'publisher_id', 'category_id',
       'ad_id', 'display_id', 'predicted_label', 'predictions_proba'],
      dtype='object')

In [97]:
drop_columns = ["document_id", "topic_id",
                        "source_id", "publisher_id", "category_id"
                ]

X_test.drop(drop_columns,axis=1,inplace=True)

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [98]:
X_test.columns

Index(['ad_id', 'display_id', 'predicted_label', 'predictions_proba'], dtype='object')

In [102]:
X_test.sort_values(by=["display_id", "ad_id", "predicted_label", "predictions_proba"], ascending=[False, False, False, False], inplace=True)



/home/nish_sharon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [103]:
X_test.drop(["predicted_label", "predictions_proba"], axis=1, inplace=True)

test_group_by = X_test.groupby("display_id")["ad_id"].apply(list)



/home/nish_sharon/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [104]:
submission_dict = {}

# create a dictionary to hold display ids and unique ad ids
for index, row in test_group_by.iteritems():
    unique_lst = []
    for i in row:
        if i not in unique_lst:
            unique_lst.append(i)
    submission_dict[index] = unique_lst

# write to the submission file
with open("submission_rf.csv", "w") as submission_file_handle:
    file_writer = csv.writer(submission_file_handle, delimiter=',', quoting=csv.QUOTE_NONE, quotechar='', lineterminator='\n')
    file_writer.writerow(["display_id","ad_id"])
    for key, value in submission_dict.items():
        ad_id_str = ""
        for i, v in enumerate(value):
            if i != 0:
                ad_id_str += " " + str(v)
            else:
                ad_id_str += str(v)

        file_writer.writerow([key, ad_id_str])

In [82]:
import numpy as np


,document_id,topic_id,source_id,publisher_id,category_id,ad_id,display_id
0,False,False,False,True,False,False,False
1,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False
5,False,False,False,True,False,False,False
6,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False
8,False,False,False,True,False,False,False
9,False,False,False,False,False,False,False


In [83]:
X_test.fillna(X_test.mean(), inplace=True)

/home/nish_sharon/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,document_id,topic_id,source_id,publisher_id,category_id,ad_id,display_id
0,1051283.0,26.0,6452.0,619.053174,1206.0,66758,16874594
1,1358132.0,163.0,8484.0,619.053174,1805.0,150083,16874594
2,1292723.0,143.0,3831.0,462.000000,1100.0,162754,16874594
3,1083829.0,131.0,10710.0,619.053174,1805.0,170392,16874594
4,1433954.0,8.0,1572.0,78.000000,1702.0,172888,16874594
5,1151028.0,16.0,4994.0,619.053174,1702.0,180797,16874594
6,21135.0,125.0,1871.0,156.000000,1405.0,8846,16874595
7,859959.0,230.0,7437.0,920.000000,1403.0,30609,16874595
8,1211785.0,192.0,8060.0,619.053174,1808.0,143982,16874595
9,635693.0,237.0,676.0,67.000000,1209.0,11430,16874596


In [ ]:
features_columns = ["document_id", "topic_id", "source_id", "publisher_id", 
                    "category_id", "ad_id", "display_id"]

X_test = test_merged[features_columns]

In [74]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32225162 entries, 0 to 32225161
Data columns (total 7 columns):
document_id     float64
topic_id        float64
source_id       float64
publisher_id    float64
category_id     float64
ad_id           int64
display_id      int64
dtypes: float64(5), int64(2)
memory usage: 1.9 GB


In [86]:
predictions_proba

array([[ 0.52076555,  0.47923445],
       [ 0.50625103,  0.49374897],
       [ 0.46483845,  0.53516155],
       ..., 
       [ 0.49089897,  0.50910103],
       [ 0.52213067,  0.47786933],
       [ 0.54819611,  0.45180389]])

In [89]:
drop_columns = ["document_id", "topic_id",
                "source_id", "publisher_id", "category_id", "platform",
                "traffic_source", "clicked", "geo_location"]

test_final_df.drop(drop_columns,axis=1,inplace=True)
#test_final_df.sort_values(['display_id','predicted_proba'], inplace=True, ascending=False)
#.isin(16874594,16874595,16874596,16874597,16874598)].sort('prob_Score'))


In [87]:
subm = test_final_df.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [78]:
subm.shape

(46824, 2)

In [91]:
test_final_df.drop("category_confidence_level",axis=1,inplace=True)

In [93]:
test_final_df.sort_values(['display_id','predicted_proba'], inplace=True, ascending=False)

In [94]:
test_final_df.drop(["predicted_label", "predicted_proba"], axis=1, inplace=True)

In [95]:
subm = test_final_df.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()

In [96]:
subm.to_csv(r'submission_file_rf_1_3.csv', index=False)
print("done")

done
